In [35]:
# default_exp docments

# Docments

> Document parameters using comments.

In [36]:
#export
from __future__ import annotations

import re
from tokenize import tokenize,COMMENT
from ast import parse,FunctionDef,AnnAssign
from io import BytesIO
from textwrap import dedent
from types import SimpleNamespace
from inspect import getsource,isfunction,isclass,signature,Parameter
from dataclasses import dataclass, is_dataclass
from fastcore.utils import *

from fastcore import docscrape
from inspect import isclass,getdoc

In [37]:
#hide
from nbdev.showdoc import *

`docments` provides programmatic access to comments in function parameters and return types. It can be used to create more developer-friendly documentation, CLI, etc tools.

## Why?

Without docments, if you want to document your parameters, you have to repeat param names in docstrings, since they're already in the function signature. The parameters have to be kept synchronized in the two places as you change your code. Readers of your code have to look back and forth between two places to understand what's happening. So it's more work for you, and for your users.

Furthermore, to have parameter documentation formatted nicely without docments, you have to use special magic docstring formatting, often with [odd quirks](https://stackoverflow.com/questions/62167540/why-do-definitions-have-a-space-before-the-colon-in-numpy-docstring-sections), which is a pain to create and maintain, and awkward to read in code. For instance, using [numpy-style documentation](https://numpydoc.readthedocs.io/en/latest/format.html):

In [38]:
def add_np(a:int, b:int=0)->int:
    """The sum of two numbers.
    
    Used to demonstrate numpy-style docstrings.

Parameters
----------
a : int
    the 1st number to add
b : int
    the 2nd number to add (default: 0)

Returns
-------
int
    the result of adding `a` to `b`"""
    return a+b

By comparison, here's the same thing using docments:

In [39]:
def add(
    a:int, # the 1st number to add
    b=0,   # the 2nd number to add
)->int:    # the result of adding `a` to `b`
    "The sum of two numbers."
    return a+b

## Numpy docstring helper functions

`docments` also supports numpy-style docstrings, or a mix or numpy-style and docments parameter documentation. The functions in this section help get and parse this information.

In [40]:
#export
def docstring(sym):
    "Get docstring for `sym` for functions and classes"
    if isinstance(sym, str): return sym
    res = getdoc(sym)
    if not res and isclass(sym): res = getdoc(sym.__init__)
    return res or ""

In [41]:
test_eq(docstring(add), "The sum of two numbers.")

In [42]:
#export
def parse_docstring(sym):
    "Parse a numpy-style docstring in `sym`"
    docs = docstring(sym)
    return AttrDict(**docscrape.NumpyDocString(docstring(sym)))

In [43]:
parse_docstring(add_np)

```json
{ 'Extended': 'Used to demonstrate numpy-style docstrings.',
  'Parameters': { 'a': Parameter(name='a', type='int', desc=['the 1st number to add']),
                  'b': Parameter(name='b', type='int', desc=['the 2nd number to add (default: 0)'])},
  'Returns': Parameter(name='', type='int', desc=['the result of adding `a` to `b`']),
  'Summary': 'The sum of two numbers.'}
```

## Usage

In [44]:
#export
def get_dcls_source(s):
    "Check if dataclass `s` has source code and returns it."
    return getsource(s) if not s.__module__ == '__main__' else ''

def get_source(s):
    'Get source code for function or dataclass `s`'
    return getsource(s) if isfunction(s) else get_dcls_source(s) if is_dataclass(s) and isclass(s) else s

def _parses(s):
    "Parse Python code in string or function or dataclass object `s`"
    return parse(dedent(get_source(s)))

def _tokens(s):
    "Tokenize Python code in string or function or dataclass object `s`"
    s = get_source(s)
    return tokenize(BytesIO(s.encode('utf-8')).readline)

_clean_re = re.compile('^\s*#(.*)\s*$')
def _clean_comment(s):
    res = _clean_re.findall(s)
    return res[0] if res else None

def _param_locs(s, returns=True):
    "`dict` of parameter line numbers to names"
    body = _parses(s).body
    if len(body)!=1: return None
    defn = body[0]
    if isinstance(defn, FunctionDef):
        res = {arg.lineno:arg.arg for arg in defn.args.args}
        if returns and defn.returns: res[defn.returns.lineno] = 'return'
        return res
    elif is_dataclass(s):
        res = {arg.lineno:arg.target.id for arg in defn.body if isinstance(arg, AnnAssign)}
        return res
    else:
        return None

In [45]:
#export
empty = Parameter.empty

In [46]:
#export
def _get_comment(line, arg, comments, parms):
    if line in comments: return comments[line].strip()
    line -= 1
    res = []
    while line and line in comments and line not in parms:
        res.append(comments[line])
        line -= 1
    return dedent('\n'.join(reversed(res))) if res else None

def _get_full(anno, name, default, docs):
    if anno==empty and default!=empty: anno = type(default)
    return AttrDict(docment=docs.get(name), anno=anno, default=default)

In [47]:
#export
def _merge_doc(dm, npdoc):
    if not npdoc: return dm
    if not dm.anno or dm.anno==empty: dm.anno = npdoc.type
    if not dm.docment: dm.docment = '\n'.join(npdoc.desc)
    return dm

def _merge_docs(dms, npdocs):
    npparams = npdocs['Parameters']
    params = {nm:_merge_doc(dm,npparams.get(nm,None)) for nm,dm in dms.items()}
    if 'return' in dms: params['return'] = _merge_doc(dms['return'], npdocs['Returns'])
    return params

In [52]:
#export
def docments(s, full=False, returns=True, eval_str=False):
    "`dict` of parameter names to 'docment-style' comments in function or string `s`"
    nps = parse_docstring(s)
    if isclass(s) and not is_dataclass(s): s = s.__init__ # Constructor for a class
    comments = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
    parms = _param_locs(s, returns=returns) or {}
    docs = {arg:_get_comment(line, arg, comments, parms) for line,arg in parms.items()}

    if isinstance(s,str): s = eval(s)
    sig = signature(s)
    res = {arg:_get_full(p.annotation, p.name, p.default, docs) for arg,p in sig.parameters.items()}
    if returns: res['return'] = _get_full(sig.return_annotation, 'return', empty, docs)
    res = _merge_docs(res, nps)
    if eval_str:
        hints = type_hints(s)
        for k,v in res.items():
            if k in hints: v['anno'] = hints.get(k)
    if not full: res = {k:v['docment'] for k,v in res.items()}
    return AttrDict(res)

In [68]:
@dataclass
class Doc:
    title:str # the title of the doc
    params:dict # the parameters of the doc
    ret:str # the return of the doc
    doc:str

# docments(Doc), signature(Doc), Doc.__dict__

# import test_dataclass

# _dcls_param_locs(getsource(test_dataclass.Point))
# docments(test_dataclass.Module), signature(test_dataclass.Module)
# p = test_dataclass.Points(1)
# p.y = 2

({'title': None, 'params': None, 'ret': None, 'doc': None, 'return': None},
 <Signature (title: 'str', params: 'dict', ret: 'str', doc: 'str') -> None>,
 mappingproxy({'__module__': '__main__',
               '__annotations__': {'title': 'str',
                'params': 'dict',
                'ret': 'str',
                'doc': 'str'},
               '__dict__': <attribute '__dict__' of 'Doc' objects>,
               '__weakref__': <attribute '__weakref__' of 'Doc' objects>,
               '__doc__': "Doc(title: 'str', params: 'dict', ret: 'str', doc: 'str')",
               '__dataclass_params__': _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False),
               '__dataclass_fields__': {'title': Field(name='title',type='str',default=<dataclasses._MISSING_TYPE object at 0x7f950b9c52d0>,default_factory=<dataclasses._MISSING_TYPE object at 0x7f950b9c52d0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_typ

In [17]:
signature(test_dataclass.Module)

<Signature (n_layers: int = 2, n_units: int = 256, parent: Union[Type[flax.linen.module.Module], Type[flax.core.scope.Scope], Type[flax.linen.module._Sentinel], NoneType] = <flax.linen.module._Sentinel object at 0x7f94ab72a950>, name: str = None) -> None>

The returned `dict` has parameter names as keys, docments as values. The return value comment appears in the `return`, unless `returns=False`. Using the `add` definition above, we get:

In [16]:
docments(test_dataclass.Module.__call__)

{2: 'x'}


```json
{'return': None, 'self': None, 'x': None}
```

In [ ]:
docments(add)

```json
{ 'a': 'the 1st number to add',
  'b': 'the 2nd number to add',
  'return': 'the result of adding `a` to `b`'}
```

If you pass `full=False`, the values are `dict` of defaults, types, and docments as values. Note that the type annotation is inferred from the default value, if the annotation is empty and a default is supplied.

In [ ]:
docments(add, full=True)

```json
{ 'a': { 'anno': 'int',
         'default': <class 'inspect._empty'>,
         'docment': 'the 1st number to add'},
  'b': { 'anno': <class 'int'>,
         'default': 0,
         'docment': 'the 2nd number to add'},
  'return': { 'anno': 'int',
              'default': <class 'inspect._empty'>,
              'docment': 'the result of adding `a` to `b`'}}
```

In [ ]:
docments(add, full=True)

```json
{ 'a': { 'anno': 'int',
         'default': <class 'inspect._empty'>,
         'docment': 'the 1st number to add'},
  'b': { 'anno': <class 'int'>,
         'default': 0,
         'docment': 'the 2nd number to add'},
  'return': { 'anno': 'int',
              'default': <class 'inspect._empty'>,
              'docment': 'the result of adding `a` to `b`'}}
```

To evaluate stringified annotations (from python 3.10), use `eval_str`:

In [ ]:
docments(add, full=True, eval_str=True)['a']

```json
{ 'anno': <class 'int'>,
  'default': <class 'inspect._empty'>,
  'docment': 'the 1st number to add'}
```

If you need more space to document a parameter, place one or more lines of comments above the parameter, or above the return type. You can mix-and-match these docment styles:

In [ ]:
def add(
    # The first operand
    a:int,
    # This is the second of the operands to the *addition* operator.
    # Note that passing a negative value here is the equivalent of the *subtraction* operator.
    b:int,
)->int: # The result is calculated using Python's builtin `+` operator.
    "Add `a` to `b`"
    return a+b

In [ ]:
docments(add)

```json
{ 'a': 'The first operand',
  'b': 'This is the second of the operands to the *addition* operator.\n'
       'Note that passing a negative value here is the equivalent of the '
       '*subtraction* operator.',
  'return': "The result is calculated using Python's builtin `+` operator."}
```

You can also use docments with classes and methods:

In [ ]:
class Adder:
    "An addition calculator"
    def __init__(self,
        a:int, # First operand
        b:int, # 2nd operand
    ): self.a,self.b = a,b
    
    def calculate(self
                 )->int: # Integral result of addition operator
        "Add `a` to `b`"
        return a+b

In [ ]:
docments(Adder)

```json
{'a': 'First operand', 'b': '2nd operand', 'return': None, 'self': None}
```

In [ ]:
docments(Adder.calculate)

```json
{'return': 'Integral result of addition operator', 'self': None}
```

docments can also be extracted from numpy-style docstrings:

In [ ]:
print(add_np.__doc__)

The sum of two numbers.
    
    Used to demonstrate numpy-style docstrings.

Parameters
----------
a : int
    the 1st number to add
b : int
    the 2nd number to add (default: 0)

Returns
-------
int
    the result of adding `a` to `b`


In [ ]:
docments(add_np)

```json
{ 'a': 'the 1st number to add',
  'b': 'the 2nd number to add (default: 0)',
  'return': 'the result of adding `a` to `b`'}
```

You can even mix and match docments and numpy parameters:

In [ ]:
def add_mixed(a:int, # the first number to add
              b
             )->int: # the result
    """The sum of two numbers.

Parameters
----------
b : int
    the 2nd number to add (default: 0)"""
    return a+b

In [ ]:
docments(add_mixed, full=True)

```json
{ 'a': { 'anno': 'int',
         'default': <class 'inspect._empty'>,
         'docment': 'the first number to add'},
  'b': { 'anno': 'int',
         'default': <class 'inspect._empty'>,
         'docment': 'the 2nd number to add (default: 0)'},
  'return': { 'anno': 'int',
              'default': <class 'inspect._empty'>,
              'docment': 'the result'}}
```

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_basics.ipynb.
Converted 02_foundation.ipynb.
Converted 03_xtras.ipynb.
Converted 03a_parallel.ipynb.
Converted 03b_net.ipynb.
Converted 04_dispatch.ipynb.
Converted 05_transform.ipynb.
Converted 06_docments.ipynb.
Converted 07_meta.ipynb.
Converted 08_script.ipynb.
Converted index.ipynb.
Converted parallel_win.ipynb.
